# Modelo ML Supervisado detector Anomalias en intraestructura en la nube

# I. Descripción del problema e inspección del conjunto de datos

## 1. Descripción del problema y conjuto de datos

El objetivo es identificar automáticamente, basándose en un conjunto de métricas operacionales (como el uso de CPU, memoria, tráfico de red, y consumo de energía), cuándo una VM se está comportando de manera anómala o inusual, en lugar de su patrón de comportamiento normal.

**Finalidad**: Este tipo de detección es crucial para la monitorización del rendimiento, el mantenimiento predictivo, y la fiabilidad general de los servicios en la nube, permitiendo identificar fallos de hardware, software o posibles incidentes de seguridad de forma temprana.

**Aproximación**: El dataset proporciona una etiqueta binaria ("Anomaly status") para cada conjunto de métricas, lo que permite el desarrollo y la evaluación de modelos de Machine Learning supervisado para clasificar si un estado de la VM es normal o anómalo

El conjunto de datos proviene de un dataset de Kanggle https://www.kaggle.com/datasets/sandhyapeesara/cloud-anomaly-data

### Descripción de Variables

| Variable | Tipo de Dato (Implícito) | Descripción Mejorada |
| :--- | :--- | :--- |
| **vm\_id** | Categórico / ID | **Identificador único** de la Máquina Virtual (VM) en el entorno de la nube. Permite rastrear el rendimiento individual de cada instancia. |
| **timestamp** | Temporal | **Marca de tiempo** exacta de la medición. Es fundamental para el análisis de **series de tiempo** y la detección de patrones temporales o estacionalidad. |
| **cpu\_usage** | Numérico (Porcentaje) | **Utilización del Procesador Central (CPU)**, expresada como un porcentaje (0-100%). Mide la carga de trabajo de la VM. |
| **memory\_usage** | Numérico (Porcentaje) | **Utilización de la Memoria RAM**, expresada como un porcentaje (0-100%). Indica la cantidad de memoria activa consumida por los procesos. |
| **network\_traffic** | Numérico | **Volumen de Tráfico de Red** (entrada/salida) de la VM. Medido en una escala relativa (0-1000); podría representar bytes/segundo o paquetes/segundo. |
| **power\_consumption** | Numérico | **Consumo de Energía** de la VM en la unidad de tiempo. Medido en una escala relativa (0-500); la unidad física podría ser Vatios (W). |
| **num\_executed\_instructions** | Numérico (Contador) | **Número de Instrucciones** ejecutadas por la CPU durante el intervalo de medición. Mide la productividad computacional. |
| **execution\_time** | Numérico | **Tiempo de Ejecución** de las tareas o procesos en la VM. Medido en una escala relativa (0-100); podría representar milisegundos o un tiempo de respuesta normalizado. |
| **energy\_efficiency** | Numérico (Ratio) | **Eficiencia Energética**. Un valor normalizado entre 0 y 1, que generalmente se calcula como una razón entre el rendimiento (ej. instrucciones ejecutadas) y el consumo de energía. |
| **task\_type** | Categórico | **Tipo de Tarea** principal que se está ejecutando en la VM en ese momento. Categorías clave: **'io'** (Intensiva en Entrada/Salida), **'network'** (Intensiva en Red), **'other'**. |
| **task\_priority** | Categórico Ordinal | **Nivel de Prioridad** asignado a la tarea en curso. Categorías: **'low'** (Baja), **'high'** (Alta), **'other'** (Otras o media/variable). |
| **task\_status** | Categórico | **Estado Actual de la Tarea** medida. Categorías: **'waiting'** (En espera de recursos), **'running'** (Actualmente en ejecución), **'other'** (Otros estados, como completada o fallida). |
| **Anomaly status** | Binario (Target) | **Estado de Anomalía**. Esta es la **variable objetivo** (`target`). Un valor de **1** indica que el comportamiento de la VM se considera **Anómalo**, y **0** indica un comportamiento **Normal**. |

### Contenido Dataset

### Import Libraries

In [ ]:
# Install libraries
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


### Import Dataset

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "Cloud_Anomaly_Dataset.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "sandhyapeesara/cloud-anomaly-data",
  file_path,
)

df.info()

#### Resumen Estadistico

In [ ]:
df.describe()

### Identificación de datos unicos

In [ ]:
df.nunique()

Preliminarmente no se puede descartar ningura variable por ser constate

#### Identificación de valores nulos

In [ ]:
df.isna().sum()

Se debe tener el cuenta las variables 'cpu_usage', 'memory_usage', 'network_traffic', 'power_comsuption', 'num_excecuted_instrucitons', 'excecution_time', energy_efficiency','task_type', 'trask_priority'y task_status' que cuentan datos nulos, donde se debe aplicacar una estrategía de corrección de los mismos

### Histogramas y diagramas de cajas para valores numericos

In [ ]:
# Variables numéricas para análisis
numerical_vars = ['cpu_usage', 'memory_usage', 'network_traffic', 'power_consumption', 'num_executed_instructions',
'execution_time', 'energy_efficiency']


#### Histogramas

In [ ]:


# Generar histogramas
df_numeric = df.select_dtypes(include=['float64'])

df_numeric.hist(edgecolor='black', linewidth=1.2, bins=30)
fig = plt.gcf()
fig.set_size_inches(15, 15)
plt.suptitle('Distribución de variables numéricas', fontsize=16)
plt.show()



Basado en los diagramas generados, podemos observar que todos siguen una distribución uniforme. Esto significa que todos los valores dentro del rando de la variable tienen aproximadamente la misma probabilidad de ocurrir. Lo que implica analisar las variables desde otro punto de vista

#### Diagramas de cajas y bigotes

In [ ]:

df_numeric = df.select_dtypes(include=['float64'])
numerical_vars = df_numeric.columns
n_vars = len(numerical_vars)

n_cols = 3 
n_rows = math.ceil(n_vars / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten()

for i, var in enumerate(numerical_vars):
    ax = axes[i]
    sns.boxplot(x=df[var], ax=ax)
    ax.set_title(f'{var}')
    ax.set_xlabel('') # Opcional: limpia la etiqueta x para no ser redundante

for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Diagramas de cajas y bigotes', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

Podemos observar que no hay puntos fuera de los bigotes, que los bigotes lleguen a los extremas es concecuencia de la distribución uniforme

Aunque la distribuición total parezca uniforme, es muy probable que las distribuciones para los datos normales y anómalos sean diferentes entre sí.

Por tanto deberiamos visualizar por Categoría de Anomalía

### Visualizar por Categoría de Anomalía
Generar los gráficos, pero esta vez, separando los datos por el valor de la columna **Anomaly status**. Esto nos permitirá comparar la distribucón de los casos normales vs los casos anómalos

In [ ]:
df_numeric = df.select_dtypes(include=['float64'])
numerical_vars = df_numeric.columns
n_vars = len(numerical_vars)

n_cols = 3 
n_rows = math.ceil(n_vars / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten()

for i, var in enumerate(numerical_vars):
    ax = axes[i]
    sns.histplot(data=df, x=var, hue='Anomaly status', kde=True, palette='viridis', ax=ax)
    ax.set_title(f'Distribución de {var}')
    ax.set_xlabel('') # Opcional: limpia la etiqueta x para no ser redundante

for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Distribución de variables numéricas (Normal vs. Anómalo)', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()


#### Análisis de Distribuciones por Estado de Anomalía

Al visualizar las distribuciones de las variables numéricas segmentadas por la variable objetivo (Anomaly status), se
revelan patrones cruciales para la construcción del modelo:

* Desbalance de Clases Significativo: Se confirma una característica común en problemas de detección de anomalías: un
    fuerte desbalance de clases. El conjunto de datos contiene una proporción considerablemente mayor de instancias
    normales (0) en comparación con las anómalas (1). Este factor es crítico y deberá ser gestionado antes de la fase de
    entrenamiento mediante balanceo

* Patrón en `network_traffic`: Esta variable demuestra ser un fuerte indicador de anomalías. Se observa una alta
    concentración de instancias anómalas en el rango de valores altos (aproximadamente 950-1000). Por el contrario, las
    instancias normales son muy infrecuentes en este mismo rango, lo que sugiere que un tráfico de red excepcionalmente
    alto es un síntoma claro de un comportamiento anómalo.

* Patrón en `energy_efficiency`: De manera similar, la eficiencia energética es un predictor clave. Los valores bajos de
    energy_efficiency (en el intervalo de 0.0 a 0.3) están fuertemente correlacionados con la ocurrencia de anomalías. Las
    instancias normales, en cambio, tienden a presentar una eficiencia energética más alta y muestran una frecuencia muy
    baja en este rango.

#### Conclusión del Análisis Visual:

El análisis exploratorio revela que, aunque las distribuciones generales de las variables pueden parecer uniformes,
existen diferencias claras y medibles en las distribuciones cuando se separan por clase (normal vs. anómela). Variables
como network_traffic y energy_efficiency son especialmente prometedoras y probablemente tendrán un alto poder predictivo
en los modelos de machine learning que se desarrollarán

### Visualizacón de Variables Categóricas
Para entender si las difenretes categorías se relacionan con el estado de anomalía

Para las variables categóricas como `task_type`, `task_priority` y `task_status`, los diagramas de barras son la herramienta ideal. Nos permitiran ver la frecuancia absoluta o relativa de cada categoría

Al igual que el analisis previo, es fundamental analizar estas distrivuciones separados por el `anomaly_status` para identificar si ciertas categorías están más asociadas con el comportamietno anómalo

In [ ]:
# Seleccionar las variables categóricas a visualizar
categorical_vars = ['task_type', 'task_priority', 'task_status']

# Generar diagramas de barras para cada variable categórica, separados por Anomaly status
for var in categorical_vars:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=var, hue='Anomaly status', palette='viridis')
    plt.title(f'Frecuencia de {var} por Estado de Anomalía')
    plt.xlabel(var)
    plt.ylabel('Conteo')
    plt.show()

- Análisis de `task_status`:
    - En el caso de las anomalías, se observa una frecuencia ligeramente superior de instancias en el estado completed en
       comparación con los estados running y waiting, los cuales presentan frecuencias muy similares entre sí.
    - Para las instancias normales, la categoría completed exhibe una frecuencia marginalmente inferior a las otras dos
      categorías (running y waiting).
    - Implicación: Este patrón sugiere que, aunque el estado completed podría parecer benigno, una proporción
      relativamente mayor de anomalías se manifiesta en este estado. Esto indica que el task_status podría ser un factor
      discriminatorio, aunque sutil, para la detección de anomalías, y su interacción con otras variables podría ser
      relevante.

- Análisis de `task_type` y `task_priority`: Para estas variables, la distribución de frecuencias de sus categorías es
    similar tanto para las instancias normales como para las anómalas, manteniendo la proporción general de desbalance de
    clases observada en el dataset. Esto sugiere que, de forma aislada, task_type y task_priority no presentan un poder
    discriminatorio directo significativo para identificar anomalías. Sin embargo, podrían ser relevantes en combinación
    con otras características o para contextualizar los eventos.



### Analisis de correlación
El análisis de correlación es un paso crucial para entender las relaciones entre las variables numéricas, y es fundamental para la selección de características

#### Analisis de correlación con Mapa de Calor

Una Matriz de correlación nos mmostrarás el coeficiente de correlación (Pearson) entre cada par de variables númericas. Un mapa de calor es una excelente manera de visualizar esta matriz

In [ ]:
numerical_vars_for_corr = [
    'cpu_usage',
    'memory_usage',
    'network_traffic',
    'power_consumption',
    'num_executed_instructions',
    'execution_time',
    'energy_efficiency',
    'Anomaly status' # Incluimos la variable objetivo
]

# Calcular la matriz de correlación
correlation_matrix = df[numerical_vars_for_corr].corr()

# Generar el mapa de calor
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlación de Variables Numéricas')
plt.show()

#### Análisis de Correlación

El análisis del mapa de calor revela los siguientes puntos clave:

- Principal Predictor Lineal (`energy_efficiency`): La variable energy_efficiency muestra la correlación más
    significativa con Anomaly status, con un coeficiente de -0.22. Esta correlación negativa moderada indica que una menor
    eficiencia energética está linealmente asociada con una mayor probabilidad de anomalía. Este resultado cuantitativo
    refuerza los hallazgos del análisis visual, donde se observó una alta frecuencia de anomalías en los rangos bajos de
    esta variable.

- Correlación Débil pero Relevante (`network_traffic`): La variable network_traffic presenta una correlación positiva
    débil de 0.04 con la variable objetivo. Aunque el valor es bajo, es el segundo más relevante y sugiere que un aumento
    en el tráfico de red tiene una ligera asociación lineal con la ocurrencia de anomalías, lo cual también es consistente
    con el análisis visual previo.

- Baja Correlación de Otras Variables: Las demás variables numéricas (cpu_usage, memory_usage, power_consumption, etc.)
    exhiben coeficientes de correlación muy cercanos a cero con Anomaly status. Esto implica que no poseen una relación
    lineal fuerte con la variable objetivo de forma individual.

- Ausencia de Multicolinealidad: Se observa una correlación extremadamente baja entre las variables predictoras. La
    ausencia de multicolinealidad es beneficiosa, ya que indica que las características son en gran medida independientes y
    no aportan información redundante, lo que simplifica la interpretación del modelo.

#### Conclusión del Análisis de Correlación:

El análisis de correlación posiciona a energy_efficiency y, en menor medida, a network_traffic como las características
numéricas con mayor poder predictivo lineal. Estos hallazgos serán fundamentales para la etapa de selección de
características y la construcción del modelo.


# II. Diseño de Experimentos y Recolección de Resultados

## 3. Estrategia de Preparación de Datos.

#### Corrigiendo datos faltantes

La estrategía que consideramos para rellenar los datos faltantes es por la media, ya que en analisis previo observamos un comportameindo uniforme, la mediana es una medida robusta para este caso, ya que no se va a ver afectada por valores extremos

Para las variables categóricas (task_type, task_priority, task_status), no podemos calcular una media o mediana. La estrategia
estándar y más lógica es usar la moda, que es el valor que aparece con más frecuencia en la columna. De esta manera, asignamos la
categoría más probable al dato faltante.


In [ ]:
# Imputación de valores nulos para variables numéricas con la mediana
numerical_cols_with_na = ['cpu_usage', 'memory_usage', 'network_traffic', 'power_consumption', 'num_executed_instructions',
'execution_time', 'energy_efficiency']

for col in numerical_cols_with_na:
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)

# Imputación de valores nulos para variables categóricas con la moda
categorical_cols_with_na = ['task_type', 'task_priority', 'task_status']

for col in categorical_cols_with_na:
    mode_value = df[col].mode()[0]
    df[col] = df[col].fillna(mode_value)


df.isna().sum()

#### Codificación de variables categoricas
Los modelos de ML no pueden trabajar directamente con texto (como 'low', 'high', 'io'). Necesitamos convertir todas las variables a formato numérico.


Tenemos dos tipos de variables categóricas que requieren un tratamiento diferente:

A. Variables Nominales (`task_type`, `task_status`)
Estas no tienen un orden inherente. Usaremos One-Hot Encoding, que crea nuevas columnas binarias (0 o 1) para cada categoría.


In [ ]:

# Aplicar One-Hot Encoding
df = pd.get_dummies(df, columns=['task_type', 'task_status'], drop_first=True)

B. Variable Ordinal (`task_priority`)
Esta variable sí tiene un orden ('low' < 'other' < 'high'). Aquí, asignaremos un número a cada categoría para preservar esa jerarquía.

In [ ]:

# Definir el orden y mapear
priority_mapping = {'low': 0, 'medium': 1, 'high': 2}
df['task_priority'] = df['task_priority'].map(priority_mapping)
print(df['task_priority'].unique())

#### Separar nuestras variable objetivo y variables caracteristicas

Separaremos nuestro DataFrame en la variable objetivo (y) que en nuestro caso es `Anomaly status` y las caracteristicas (x) que usaremos para la predicción. También eliminaremos las columnas que no son útiles para el modelo, como `vm_id` y `timestamp`

In [ ]:
# 'y' es nuestra variable objetivo
y = df['Anomaly status']

# 'X' son todas las demás columnas, excepto las que no aportan información predictiva
X = df.drop(columns=['Anomaly status', 'vm_id', 'timestamp'])

print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", y.shape)

#### División en Conjuntos de Entrenamiento y Prueba

Dividimos los datos en un conjunto para entrenar el modelo y otro para evaluarlo de forma imaparcial

In [ ]:

# Dividir los datos (70% entrenamiento, 30% prueba)
# stratify=y asegura que la proporción de anomalías sea la misma en ambos conjuntos, lo cual es clave en datasets desbalanceados.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("Tamaño de X_train:", X_train.shape)
print("Tamaño de X_test:", X_test.shape)

#### Balanceo de clases

Ya que hemos dividido los datos correctamente, podemos abordar el problema del desbalance

No solo debemos dubplicar los casos de anomalias, sino que cree nuevas muestras sintéticas que sean similares a las existentes, para este proposito usaremos la tecnica **SMOTE SMOTE (Synthetic Minority Over-sampling Technique)**

In [ ]:
# Inicializar SMOTE
smote = SMOTE(random_state=42)

# Aplicar SMOTE SOLO a los datos de entrenamiento
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.countplot(x=y_train, hue=y_train, ax=axes[0], palette='viridis', legend=False)
axes[0].set_title('Antes de SMOTE (Datos de Entrenamiento)')
axes[0].set_xlabel('Estado de Anomalía')
axes[0].set_ylabel('Conteo')
for container in axes[0].containers:
    axes[0].bar_label(container)

sns.countplot(x=y_train_resampled, hue=y_train_resampled, ax=axes[1], palette='viridis', legend=False)
axes[1].set_title('Después de SMOTE (Datos de Entrenamiento)')
axes[1].set_xlabel('Estado de Anomalía')
axes[1].set_ylabel('Conteo')
for container in axes[1].containers:
    axes[1].bar_label(container)

plt.tight_layout()
plt.show()

Los datos ya se encuentra balanceados correctamente

#### Selección de Características

Después de preparar y balancear los datos, el siguiente paso es seleccionar las características más influyentes para predecir el
estado de anomalía. Usaremos métodos estadísticos para evaluar la relación de cada característica con la variable objetivo (`Anomaly
status`).

- Para las **variables numéricas**, usaremos la prueba **ANOVA F-test** (`f_classif`), que nos dirá si hay una diferencia
significativa en las medias de la característica entre las clases "Normal" y "Anómala".
- Para las **variables categóricas** (ya codificadas), usaremos la prueba **Chi-Cuadrado** (`chi2`), que evalúa la dependencia entre
la característica y la clase objetivo.

Esto nos ayudará a reducir la dimensionalidad, mejorar el rendimiento del modelo y reducir el riesgo de sobreajuste.


In [ ]:
# Identificar las columnas numéricas y categóricas en nuestro conjunto de datos procesado
# 'task_priority' se trata como numérica porque la mapeamos a números (0, 1, 2)
numerical_features = [
    'cpu_usage', 'memory_usage', 'network_traffic', 'power_consumption',
    'num_executed_instructions', 'execution_time', 'energy_efficiency', 'task_priority'
]

# Las columnas categóricas son las que se crearon con get_dummies()
categorical_features = [
    'task_type_io', 'task_type_network',
    'task_status_running', 'task_status_waiting'
]

# Es posible que los nombres de las columnas categóricas varíen ligeramente.
# Vamos a verificar y corregir los nombres si es necesario.
# El siguiente código asegura que solo usemos las columnas que realmente existen en nuestro DataFrame.

existing_categorical_features = [col for col in categorical_features if col in X_train_resampled.columns]

# Si get_dummies() usó 'other' o 'completed' como base, los nombres de las columnas serían diferentes.
# Vamos a listar todas las columnas dummies para estar seguros.
print("Columnas del DataFrame:", X_train_resampled.columns.tolist())
print("-" * 30)
print("Columnas categóricas identificadas:", existing_categorical_features)

Aplicamos ANOVA F-test a las carecteristicas númericas

In [ ]:
# 1. Configurar el selector de características para usar f_classif
# Seleccionaremos todas las características (k='all') para ver la puntuación de cada una
fs_numerical = SelectKBest(score_func=f_classif, k='all')

# 2. Aplicar el selector a nuestras variables numéricas del conjunto de entrenamiento
fs_numerical.fit(X_train_resampled[numerical_features], y_train_resampled)

# 3. Crear un DataFrame con las puntuaciones para una fácil visualización
numerical_scores = pd.DataFrame({
    'Feature': numerical_features,
    'F-Score': fs_numerical.scores_,
    'P-Value': fs_numerical.pvalues_
}).sort_values(by='F-Score', ascending=False)

print("Puntuaciones de Características Numéricas (ANOVA F-test):")
print(numerical_scores)

# 4. Visualizar las puntuaciones
plt.figure(figsize=(10, 6))
sns.barplot(x='F-Score', y='Feature', data=numerical_scores, palette='viridis')
plt.title('Importancia de Características Numéricas (F-Score)')
plt.xlabel('Puntuación F (ANOVA)')
plt.ylabel('Característica')
plt.show()

Aplicar la prueba de Chi-cuadrado a las caracteristicas categóricas

In [ ]:
# Actualizar la lista de características categóricas con los nombres confirmados
categorical_features = ['task_type_io', 'task_type_network', 'task_status_running', 'task_status_waiting']

# 1. Configurar el selector de características para usar chi2
# Seleccionaremos todas las características (k='all') para ver la puntuación de cada una
fs_categorical = SelectKBest(score_func=chi2, k='all')

# 2. Aplicar el selector a nuestras variables categóricas del conjunto de entrenamiento
fs_categorical.fit(X_train_resampled[categorical_features], y_train_resampled)

# 3. Crear un DataFrame con las puntuaciones para una fácil visualización
categorical_scores = pd.DataFrame({
    'Feature': categorical_features,
    'Chi2-Score': fs_categorical.scores_,
    'P-Value': fs_categorical.pvalues_
}).sort_values(by='Chi2-Score', ascending=False)

print("\nPuntuaciones de Características Categóricas (Chi-Cuadrado):")
print(categorical_scores)

# 4. Visualizar las puntuaciones
plt.figure(figsize=(10, 6))
sns.barplot(x='Chi2-Score', y='Feature', data=categorical_scores, palette='magma')
plt.title('Importancia de Características Categóricas (Chi-Cuadrado Score)')
plt.xlabel('Chi2 Score')
plt.ylabel('Característica')
plt.show()

#### Análisis de los Resultados

1. Características Numéricas (ANOVA F-test):

 - Muy Importantes: energy_efficiency, task_priority, y network_traffic tienen puntuaciones F-Score altísimas. Esto confirma lo que ya
   sospechábamos en el análisis visual: son, por mucho, los predictores numéricos más fuertes.
 - Medianamente Importantes: memory_usage también es claramente relevante, aunque en menor medida.
 - Poco Importantes: execution_time y power_consumption tienen una relación estadísticamente significativa (P-Value < 0.05), pero su
   puntuación F es muy baja, lo que sugiere que su poder predictivo es débil.
 - No Relevantes: num_executed_instructions y cpu_usage tienen un P-Value > 0.05. Esto significa que no podemos descartar que su
   relación con la anomalía sea producto del azar. Por lo tanto, son candidatos claros a ser eliminados.

2. Características Categóricas (Chi-Cuadrado):

 - Todas las características categóricas (task_type_io, task_type_network, task_status_running, task_status_waiting) muestran
   puntuaciones Chi2-Score muy altas y P-Values de 0.0.
 - Esto indica que todas ellas tienen una fuerte asociación con la variable objetivo y son importantes para el modelo. Las relacionadas
   con el tipo de tarea (task_type) parecen ser ligeramente más fuertes que las del estado (task_status), pero todas son muy valiosas.


Basado en estos resultados vamos a:

- Conservar todas las características categoricas
- Conservar las 6 caracteristicas númericas que mostraron una relación estadisticamente significativa (P-Value < 0.05)
- Eliminar las 2 características numéricas que no son estadísticamente significativas (num_executed_instructions y cpu_usage).

In [ ]:
# Definir la lista final de características seleccionadas
final_features = [
    # Numéricas significativas
    'energy_efficiency',
    'task_priority',
    'network_traffic',
    'memory_usage',
    'execution_time',
    'power_consumption',
    # Categóricas (todas fueron significativas)
    'task_type_io',
    'task_type_network',
    'task_status_running',
    'task_status_waiting'
]

# Crear los nuevos DataFrames solo con las características seleccionadas
X_train_selected = X_train_resampled[final_features]
X_test_selected = X_test[final_features]

print("Dimensiones del conjunto de entrenamiento después de la selección:", X_train_selected.shape)
print("Dimensiones del conjunto de prueba después de la selección:", X_test_selected.shape)
print("\nCaracterísticas finales:")
print(X_train_selected.columns.tolist())

#### Escalado de caracteristicas
Después de seleccionar las características, escalaremos sus valores para que todos tengan una distribución con media 0 y desviación estándar 1. Esto es crucial para modelos sensibles a la escala, como la Regresión Logística y las Redes Neuronales, ya que ayuda a que los algoritmos de optimización converjan mucho más rápido y de manera más estable.

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_selected)

# 3. Aplicar la MISMA transformación a los datos de prueba
X_test_scaled = scaler.transform(X_test_selected)

## 4. Experimentación y Ajuste de Parámetros
En esta fase, entrenaremos y evaluaremos los diferentes modelos de clasificación solicitados en la guía. Para cada modelo,
utilizaremos `GridSearchCV` para realizar una búsqueda exhaustiva de los mejores hiperparámetros. `GridSearchCV` utiliza validación
cruzada (`cross-validation`) para asegurar que los resultados sean robustos y evitar el sobreajuste.

El rendimiento de cada modelo se evaluará inicialmente con la métrica de `accuracy` durante la búsqueda de parámetros, y el mejor
modelo de cada tipo se evaluará finalmente sobre el conjunto de prueba (`X_test_selected`) utilizando un reporte de clasificación
completo (precisión, recall, f1-score) y una matriz de confusión.


### Modelo 1:  Regresión Logistica ( Analogo de Regresioń Multivariada)

Para nuestro caso particular usaremos este modelo que se utiliza para predecir la probabilidad de un evento binario (como es nuestro caso) basandose en uno o más predictores. Utilizando la función logística `sigmoide` para transformar los resultados en una valor entre 0 y 1


In [ ]:
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

grid_search_lr = GridSearchCV(
    estimator=LogisticRegression(random_state=42, max_iter=1000),
    param_grid=param_grid_lr,
    scoring='f1_weighted',
    cv=5,
    n_jobs=-1
)

print("Iniciando búsqueda para Regresión Logística (con datos escalados)...")
grid_search_lr.fit(X_train_scaled, y_train_resampled)

print("\nMejores hiperparámetros encontrados:")
print(grid_search_lr.best_params_)
print("\nMejor puntuación de cross-validation (accuracy):")
print(f"{grid_search_lr.best_score_:.4f}")

best_lr = grid_search_lr.best_estimator_
y_pred_lr = best_lr.predict(X_test_scaled)

print("\n--- Reporte de Clasificación en el Conjunto de Prueba (con datos escalados) ---")
print(classification_report(y_test, y_pred_lr))

print("Matriz de Confusión:")
ConfusionMatrixDisplay.from_estimator(best_lr, X_test_scaled, y_test, cmap='Blues')
plt.title('Matriz de Confusión - Regresión Logística (Datos Escalados)')
plt.show()

### Modelo 2: Árbol de Decisión

In [ ]:


# --- 1. Definir el modelo y los hiperparámetros a probar ---
param_grid_dt = {
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

# --- 2. Configurar y ejecutar GridSearchCV ---
# Usamos los datos escalados, aunque los árboles no son sensibles a la escala
grid_search_dt = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid_dt,
    scoring='f1_weighted',
    cv=5,
    n_jobs=-1
)

print("Iniciando búsqueda de hiperparámetros para Árbol de Decisión...")
grid_search_dt.fit(X_train_scaled, y_train_resampled)

# --- 3. Mostrar los mejores resultados de la búsqueda ---
print("\nMejores hiperparámetros encontrados:")
print(grid_search_dt.best_params_)
print("\nMejor puntuación de cross-validation (accuracy):")
print(f"{grid_search_dt.best_score_:.4f}")

# --- 4. Evaluar el mejor modelo en el conjunto de prueba ---
best_dt = grid_search_dt.best_estimator_
y_pred_dt = best_dt.predict(X_test_scaled)

print("\n--- Reporte de Clasificación en el Conjunto de Prueba ---")
print(classification_report(y_test, y_pred_dt))

# --- 5. Visualizar la Matriz de Confusión ---
print("Matriz de Confusión:")
ConfusionMatrixDisplay.from_estimator(best_dt, X_test_scaled, y_test, cmap='Greens')
plt.title('Matriz de Confusión - Árbol de Decisión')
plt.show()

## Modelo 3: Random Forest

In [ ]:

param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5]
}

# --- 2. Configurar y ejecutar GridSearchCV ---
grid_search_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_rf,
    scoring='f1_weighted',
    cv=5,
    n_jobs=-1
)

print("Iniciando búsqueda de hiperparámetros para Random Forest...")
grid_search_rf.fit(X_train_scaled, y_train_resampled)

# --- 3. Mostrar los mejores resultados de la búsqueda ---
print("\nMejores hiperparámetros encontrados:")
print(grid_search_rf.best_params_)
print("\nMejor puntuación de cross-validation (accuracy):")
print(f"{grid_search_rf.best_score_:.4f}")

# --- 4. Evaluar el mejor modelo en el conjunto de prueba ---
best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test_scaled)

print("\n--- Reporte de Clasificación en el Conjunto de Prueba ---")
print(classification_report(y_test, y_pred_rf))

# --- 5. Visualizar la Matriz de Confusión ---
print("Matriz de Confusión:")
ConfusionMatrixDisplay.from_estimator(best_rf, X_test_scaled, y_test, cmap='Purples')
plt.title('Matriz de Confusión - Random Forest')
plt.show()


## Modelo 4: Redes Neuronales (MLP)

El modelo que usaremos es el MLPClassifier, que es un perceptron de Scikit-learn, que es un perceptrón Multicapa. Es ideal para este tipo de problemas y muy sensible al escalado de datos.


In [ ]:
# Definir el modelo y los hiperparámetros a probar
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001],
    'max_iter': [500]
}

# 2. Configurar y ejecutar GridSearchCV
grid_search_mlp = GridSearchCV(
    estimator=MLPClassifier(random_state=42),
    param_grid=param_grid_mlp,
    scoring='f1_weighted',
    cv=5,
    n_jobs=-1
)

print("Iniciando búsqueda de hiperparámetros para Red Neuronal (MLP)...")
grid_search_mlp.fit(X_train_scaled, y_train_resampled)

# 3. Mostrar los mejores resultados
print("\\nMejores hiperparámetros encontrados:")
print(grid_search_mlp.best_params_)
print(f"\\nMejor puntuación de cross-validation (accuracy): {grid_search_mlp.best_score_:.4f}")


# 4. Evaluar el mejor modelo en el conjunto de prueba
best_mlp = grid_search_mlp.best_estimator_
y_pred_mlp = best_mlp.predict(X_test_scaled)

print("\\n--- Reporte de Clasificación en el Conjunto de Prueba ---")
print(classification_report(y_test, y_pred_mlp))

# 5. Visualizar la Matriz de Confusión
print("Matriz de Confusión:")
ConfusionMatrixDisplay.from_estimator(best_mlp, X_test_scaled, y_test, cmap='Oranges')
plt.title('Matriz de Confusión - Red Neuronal (MLP)')
plt.show()



## 5. Descripción de los experimentos de ajuste de paramentros

## 5. Analisis de Resultados Finales

# III. Comparación de Modelos entrenados y conclusiones


## 6. Comparacion del mejor modelo obtenido por cada método

## 7. Conclusiones del trabajo